In [3]:
pip install altair


  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.23.1-cp312-cp312-win_amd64.whl.metadata (4.2 kB)
Using cached altair-5.5.0-py3-none-any.whl (731 kB)
Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
   ---------------------------------------- 0.0/313.1 kB ? eta -:--:--
   - -------------------------------------- 10.2/313.1 kB ? eta -:--:--
   ------- ------------------------------- 61.4/313.1 kB 812.7 kB/s eta 0:00:01
   ------------------ --------------------- 143.4/313.1 kB 1.2 MB/s eta 0:00:01
   ---------------------------- ----------- 225.3/313.1 kB 1.4 MB/s eta 0:00:01
   ------------------------------------- -- 297.0/313.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 313.1/313.1 kB 1.4 MB/s et

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
import altair as alt

In [5]:
alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('colab')

RendererRegistry.enable('colab')

Kode di atas memiliki dua buah sintaks yang berperan sebagai ground rules ketika menggunakan Altair sebagai library visualisasi data. Mari kita jabarkan satu per satu.

1. alt.data_transformers.enable('default', max_rows=None): baris ini mengatur transformer data Altair untuk digunakan dalam notebook. Transformer data ini mengubah data mentah menjadi format yang dapat digunakan oleh Altair untuk membuat visualisasi. Dalam baris ini, kita mengatur transformer data menjadi 'default', biasanya parameter ini dapat mengubah data menjadi format yang cocok untuk visualisasi. Pengaturan max_rows=None berperan untuk menghilangkan batasan baris data yang dapat diplot. Ini bermanfaat jika kita memiliki dataset besar yang ingin kita visualisasikan sepenuhnya.
2. alt.renderers.enable('colab'): baris ini mengatur Altair agar dapat digunakan dalam Google Colab. Renderer adalah komponen yang bertanggung jawab untuk menampilkan visualisasi di lingkungan tempat kita bekerja, seperti browser atau notebook. Dalam baris ini, kita mengatur renderer menjadi 'colab' yang cocok untuk penggunaan Altair di Google Colab. Dengan mengatur renderer ini, visualisasi yang kita buat dengan Altair akan ditampilkan secara langsung di Google Colab.

In [7]:
# melaku,an downlaod data
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfile
 
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "movielens.zip")
zip_ref = zipfile.ZipFile('movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [15]:
# Load masing masing dataset (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')
 
ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# Genre pada setiap film bersifat biner.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = ['movie_id', 'title', 'release_date', "video_release_date", "imdb_url"] + genre_cols
movies = pd.read_csv('ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

In [16]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [17]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [14]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [23]:
#melakukan rekap
print('Jumlah data film: ', len(movies.movie_id.unique()))
print('Jumlah data pengunjung: ', len(users.user_id.unique()))
print('Jumlah data penayangan: ', len(ratings.user_id))

Jumlah data film:  1682
Jumlah data pengunjung:  943
Jumlah data penayangan:  100000


# eksplorasi data


In [29]:
# Pastikan nilai numerik dikonversi ke integer sebelum operasi
users["user_id"] = users["user_id"].astype(int).apply(lambda x: str(x - 1))
movies["movie_id"] = movies["movie_id"].astype(int).apply(lambda x: str(x - 1))
movies["year"] = movies["release_date"].astype(str).apply(lambda x: x.split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].astype(int).apply(lambda x: str(x - 1))
ratings["user_id"] = ratings["user_id"].astype(int).apply(lambda x: str(x - 1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))  # Konversi langsung tanpa apply


In [30]:
genre_occurences = movies[genre_cols].sum().to_dict()
genre_occurences

{'genre_unknown': 2,
 'Action': 251,
 'Adventure': 135,
 'Animation': 42,
 'Children': 122,
 'Comedy': 505,
 'Crime': 109,
 'Documentary': 50,
 'Drama': 725,
 'Fantasy': 22,
 'Film-Noir': 24,
 'Horror': 92,
 'Musical': 56,
 'Mystery': 61,
 'Romance': 247,
 'Sci-Fi': 101,
 'Thriller': 251,
 'War': 71,
 'Western': 27}

proses embeddings seperti pada modul Natural Language Processing sebelumnya. Oleh karena itu, kita perlu membuat sebuah fungsi yang dapat menggabungkan genre menjadi satu buah string. 

In [31]:
def mark_genres(movies, genres):
  def get_random_genre(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return np.random.choice(active)
  def get_all_genres(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return '-'.join(active)
  movies['genre'] = [
      get_random_genre(gs) for gs in zip(*[movies[genre] for genre in genres])]
  movies['all_genres'] = [
      get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]
 
mark_genres(movies, genre_cols)

In [32]:
movies['all_genres']

0       Animation-Children-Comedy
1       Action-Adventure-Thriller
2                        Thriller
3             Action-Comedy-Drama
4            Crime-Drama-Thriller
                  ...            
1677                        Drama
1678             Romance-Thriller
1679                Drama-Romance
1680                       Comedy
1681                        Drama
Name: all_genres, Length: 1682, dtype: object

In [33]:
movielens = ratings.merge(movies, on='movie_id').merge(users, on='user_id')


# eksplorasi data

In [35]:
#  membuat fungsi untuk format display data 

pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]
 
def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df
 
pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

1. Pengaturan Opsi Tampilan: bagian pertama dari kode, yaitu pd.options.display.max_rows dan pd.options.display.float_format bertujuan untuk mengatur tampilan default saat kita mencetak atau menampilkan DataFrame menggunakan library pandas. Dengan mengatur max_rows, kita membatasi jumlah baris yang akan ditampilkan secara default, sementara dengan float_format, kita mengatur format tampilan untuk nilai floating point dalam DataFrame.
2. Penambahan Fungsi-Fungsi Manipulasi DataFrame: bagian kedua dari kode mendefinisikan dua fungsi, yaitu mask dan flatten_cols, dan menambahkannya ke objek DataFrame dari library pandas. Ini memperluas fungsionalitas bawaan DataFrame dengan fungsi-fungsi baru yang dapat digunakan untuk manipulasi data, yaitu:
- Mask: untuk memfilter DataFrame berdasarkan kriteria yang diberikan pada kolom tertentu.
- Flatten_cols: untuk menggabungkan multi-indeks kolom pada DataFrame menjadi satu level sehingga mempermudah manipulasi dan analisis data.

In [36]:
users.describe(include="all")

,user_id,age,sex,occupation,zip_code
count,943,943.000,943,943,943
unique,943,NaN,2,21,795
top,-4,NaN,M,student,55414
freq,1,NaN,670,196,9
mean,NaN,34.052,NaN,NaN,NaN
...,...,...,...,...,...
min,NaN,7.000,NaN,NaN,NaN
25%,NaN,25.000,NaN,NaN,NaN
50%,NaN,31.000,NaN,NaN,NaN
75%,NaN,43.000,NaN,NaN,NaN


In [45]:
# Membuat filter untuk melakukan slicing data.
occupation_filter = alt.selection_multi(fields=["occupation"])
occupation_chart = alt.Chart().mark_bar().encode(
    x="count()",
    y=alt.Y("occupation:N"),
    color=alt.condition(
        occupation_filter,
        alt.Color("occupation:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).properties(width=300, height=300, selection=occupation_filter)
 
# Fungsi yang dapat membuat histogram dari data yang sudah difilter.
def filtered_hist(field, label, filter):
  base = alt.Chart().mark_bar().encode(
      x=alt.X(field, bin=alt.Bin(maxbins=10), title=label),
      y="count()",
  ).properties(
      width=300,
  )
  return alt.layer(
      base.transform_filter(filter),
      base.encode(color=alt.value('lightgray'), opacity=alt.value(.7)),
  ).resolve_scale(y='independent')

C:\Users\rasyid\AppData\Local\Temp\ipykernel_14016\2726002556.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  occupation_filter = alt.selection_multi(fields=["occupation"])


In [52]:
# dataset ratings
occupation_filter = alt.selection_multi(fields=["occupation"])
occupation_chart = alt.Chart().mark_bar().encode(
    x="count()",
    y=alt.Y("occupation:N"),
    color=alt.condition(
        occupation_filter,
        alt.Color("occupation:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).add_selection(occupation_filter) # Move selection here
# ... (Rest of your code remains the same)
# Create a chart for the count, and one for the mean.
alt.hconcat(
    filtered_hist('rating count', '# ratings / user', occupation_filter),
    filtered_hist('rating mean', 'mean user rating', occupation_filter),
    occupation_chart, data=users_ratings)

C:\Users\rasyid\AppData\Local\Temp\ipykernel_14016\1580852354.py:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  occupation_filter = alt.selection_multi(fields=["occupation"])
C:\Users\rasyid\AppData\Local\Temp\ipykernel_14016\1580852354.py:10: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(occupation_filter) # Move selection here


NameError: name 'users_ratings' is not defined

In [39]:
movies_ratings = movies.merge(
    ratings
    .groupby('movie_id', as_index=False)
    .agg({'rating': ['count', 'mean']})
    .flatten_cols(),
    on='movie_id')
genre_filter = alt.selection_multi(fields=['genre'])
genre_chart = alt.Chart().mark_bar().encode(
    x="count()",
    y=alt.Y('genre'),
    color=alt.condition(
        genre_filter,
        alt.Color("genre:N"),
        alt.value('lightgray'))
).properties(height=300).add_selection(genre_filter) # add selection here
# Display the number of ratings and average rating per movie.
alt.hconcat(
    filtered_hist('rating count', '# ratings / movie', genre_filter),
    filtered_hist('rating mean', 'mean movie rating', genre_filter),
    genre_chart,
    data=movies_ratings)

C:\Users\rasyid\AppData\Local\Temp\ipykernel_14016\212207840.py:7: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  genre_filter = alt.selection_multi(fields=['genre'])
C:\Users\rasyid\AppData\Local\Temp\ipykernel_14016\212207840.py:15: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).properties(height=300).add_selection(genre_filter) # add selection here


alt.HConcatChart(...)

In [53]:
(movies_ratings[['title', 'rating count', 'rating mean']]
 .sort_values('rating count', ascending=True)
 .head(10))

,title,rating count,rating mean
1679,Scream of Stone (Schrei aus Stein) (1991),1,2.000
1576,Baton Rouge (1988),1,1.000
1575,"Collectionneuse, La (1967)",1,1.000
1574,"Death in the Garden (Mort en ce jardin, La) (1...",1,1.000
1573,"Hungarian Fairy Tale, A (1987)",1,1.000
1571,Pharaoh's Army (1995),1,1.000
1570,Spirits of the Dead (Tre passi nel delirio) (1...,1,1.000
1569,Wend Kuuni (God's Gift) (1982),1,1.000
1578,"Woman in Question, The (1950)",1,1.000
1568,Touki Bouki (Journey of the Hyena) (1973),1,1.000


In [ ]:
# ratingnya berdasarkan user
(movies_ratings[['title', 'rating count', 'rating mean']]
 .mask('rating count', lambda x: x > 20)
 .sort_values('rating mean', ascending=False)
 .head(10))

,title,rating count,rating mean
407,Kingpin (1996),112,4.491
317,Paradise Lost: The Child Murders at Robin Hood...,298,4.466
168,Delicatessen (1991),118,4.466
482,My Fair Lady (1964),243,4.457
113,Cold Comfort Farm (1995),67,4.448
63,While You Were Sleeping (1995),283,4.445
602,Meet Me in St. Louis (1944),209,4.388
11,"Postino, Il (1994)",267,4.386
49,"Madness of King George, The (1994)",583,4.358
177,Apocalypse Now (1979),125,4.344


In [55]:
# melakukan data split

def split_dataframe(df, holdout_function=0.1):
    test = df.sample(frac = holdout_function, replace=False)
    train = df[~df.index.isin(test.index)]
    return train, test

code tersebut akan membaginya ke dalam test dan dataset
![Alt text](https://assets.cdn.dicoding.com/original/academy/dos-4b9b3290124a87917c91067e029d517620240625164233.jpeg)

In [56]:
def build_rating_sparse_tensor(ratings_df):
  indices = ratings_df[['user_id', 'movie_id']].values
  values = ratings_df['rating'].values
  return tf.SparseTensor(
      indices=indices,
      values=values,
      dense_shape=[users.shape[0], movies.shape[0]])

In [58]:
# fugsi untuk menghitung loss dengan MSE
def sparse_mean_square_error(sparse_ratings, user_embeddings, movie_embeddings):
  predictions = tf.gather_nd(
      tf.matmul(user_embeddings, movie_embeddings, transpose_b=True),
      sparse_ratings.indices)
  loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
  return loss

1. sparse_ratings: matriks SparseTensor yang telah kita buat pada kode sebelumnya dengan bentuk larik [N, M].
2. user_embeddings: sebuah Tensor padat U dengan bentuk [N, k] di mana k adalah embedding sehingga U_i adalah embedding dari pengguna i.
3. movie_embeddings: sebuah Tensor padat V dengan bentuk [M, k] di mana k adalah dimensi penyematan dimensi embedding sehingga V_j adalah embedding dari film j.

1. tf.matmul(user_embeddings, movie_embeddings, transpose_b=True): fungsi ini melakukan perkalian matriks antara embedding pengguna (user_embeddings) dan embedding film (movie_embeddings). Lalu, transpose_b=True bertugas untuk mentranspose posisi kolom dan baris dari movie_embeddings sehingga dimensi cocok untuk perkalian matriks. Hasil dari operasi ini adalah sebuah matriks di mana setiap elemen [i, j] adalah skor prediksi dari pengguna i untuk film j.
2. tf.gather_nd(..., sparse_ratings.indices): bertugas untuk mengambil elemen-elemen dari hasil perkalian matriks berdasarkan indeks yang diberikan oleh sparse_ratings.indices. Indeks ini menunjukkan posisi elemen non-nol dalam sparse_ratings yang berarti kita hanya mengambil prediksi yang relevan.
3. loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
- sparse_ratings.values: ini adalah nilai-nilai peringkat sebenarnya yang diberikan oleh pengguna untuk film.
- predictions: ini adalah prediksi peringkat yang diambil dari matriks hasil perkalian embedding.
- tf.losses.mean_squared_error(...): fungsi ini menghitung MSE antara nilai sebenarnya dan prediksi.

In [59]:
# menentukan jenis rekomendasi menggunakan colaboratice filtering
class CFModel(object):
  def __init__(self, embedding_vars, loss, metrics=None):
    self._embedding_vars = embedding_vars
    self._loss = loss
    self._metrics = metrics
    self._embeddings = {k: None for k in embedding_vars}
    self._session = None
 
  @property
  def embeddings(self):
    return self._embeddings
 
  def train(self, num_iterations=100, learning_rate=1.0, plot_results=True,
            optimizer=tf.train.GradientDescentOptimizer):
    with self._loss.graph.as_default():
      opt = optimizer(learning_rate)
      train_op = opt.minimize(self._loss)
      local_init_op = tf.group(
          tf.variables_initializer(opt.variables()),
          tf.local_variables_initializer())
      if self._session is None:
        self._session = tf.Session()
        with self._session.as_default():
          self._session.run(tf.global_variables_initializer())
          self._session.run(tf.tables_initializer())
          tf.train.start_queue_runners()
 
    with self._session.as_default():
      local_init_op.run()
      iterations = []
      metrics = self._metrics or ({},)
      metrics_vals = [collections.defaultdict(list) for _ in self._metrics]
      for i in range(num_iterations + 1):
        _, results = self._session.run((train_op, metrics))
        if (i % 10 == 0) or i == num_iterations:
          print("\r iteration %d: " % i + ", ".join(
                ["%s=%f" % (k, v) for r in results for k, v in r.items()]),
                end='')
          iterations.append(i)
          for metric_val, result in zip(metrics_vals, results):
            for k, v in result.items():
              metric_val[k].append(v)
 
      for k, v in self._embedding_vars.items():
        self._embeddings[k] = v.eval()
 
      if plot_results:
        num_subplots = len(metrics)+1
        fig = plt.figure()
        fig.set_size_inches(num_subplots*10, 8)
        for i, metric_vals in enumerate(metrics_vals):
          ax = fig.add_subplot(1, num_subplots, i+1)
          for k, v in metric_vals.items():
            ax.plot(iterations, v, label=k)
          ax.set_xlim([1, num_iterations])
          ax.legend()
      return results

In [60]:
# struktur kode agar dapat mudah dimengerti
'''
U_var = ...
V_var = ...
loss = ...
model = CFModel(U_var, V_var, loss)
model.train(iterations=100, learning_rate=1.0)
user_embeddings = model.embeddings['user_id']
movie_embeddings = model.embeddings['movie_id']
'''
def build_model(ratings, embedding_dim, init_stddev):
  # Membagi dataset menjadi data latih dan data uji.
  train_ratings, test_ratings = split_dataframe(ratings)
  # Mengubah dataset menjadi SparseTensor.
  A_train = build_rating_sparse_tensor(train_ratings)
  A_test = build_rating_sparse_tensor(test_ratings)
  # Inisialisasi embedding menggunakan distribusi normal.
  U = tf.Variable(tf.random_normal(
      [A_train.dense_shape[0], embedding_dim], stddev=init_stddev))
  V = tf.Variable(tf.random_normal(
      [A_train.dense_shape[1], embedding_dim], stddev=init_stddev))
  train_loss = sparse_mean_square_error(A_train, U, V)
  test_loss = sparse_mean_square_error(A_test, U, V)
  metrics = {
      'train_error': train_loss,
      'test_error': test_loss
  }
  embeddings = {
      "user_id": U,
      "movie_id": V
  }
  return CFModel(embeddings, train_loss, [metrics])


In [61]:
model = build_model(ratings, embedding_dim=30, init_stddev=0.5)
model.train(num_iterations=1000, learning_rate=10.)

InvalidArgumentError: Graph execution error:

Detected at node 'GatherNd_1' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "C:\Users\rasyid\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    File "C:\Users\rasyid\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    File "C:\Users\rasyid\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    File "C:\Users\rasyid\AppData\Roaming\Python\Python312\site-packages\tornado\platform\asyncio.py", line 205, in start
    File "c:\Users\rasyid\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 641, in run_forever
    File "c:\Users\rasyid\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 1987, in _run_once
    File "c:\Users\rasyid\AppData\Local\Progra